[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AlirezaMorsali/MLP-Attention/blob/am/farsi/farsi/AI-Poet-inference.ipynb)

# Functions


In [1]:
%%capture
!pip install wget
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import os
from google.colab import drive
import pickle
import wget


In [2]:
experiment_name = "Final"
result_path = f"results/{experiment_name}"
if not os.path.exists(result_path):
    os.makedirs(result_path)
batch_size = 64  # how many independent sequences will we process in parallel?
block_size = 256  # what is the maximum context length for predictions?
max_iters = 1000
eval_interval = 50
learning_rate = 0.0003
device = "cuda" if torch.cuda.is_available() else "cpu"
eval_iters = 200
n_embd = 384
n_head = 3
n_layer = 3
n_hidden_layers = 1
dropout = 0.2
hidden_size = block_size

batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 64 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 1000
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 128
n_head = 6
n_layer = 6
dropout = 0.0

poet_names = ["ferdousi", "saadi", "hafez", "moulavi", "khayyam"]

url = "https://raw.githubusercontent.com/AlirezaMorsali/MLP-Attention/am/farsi/farsi/checkpoints"
for poet_name in poet_names:
  model_name = f"{poet_name}.pth"
  dict_name = f"{poet_name}_dictionary.pkl"
  wget.download(f"{url}/{model_name}", model_name)
  wget.download(f"{url}/{dict_name}", dict_name)

In [3]:
def encode(s):
    return [stoi[c] for c in s]  # encoder: take a string, output a list of integers


def decode(l):
    return "".join(
        [itos[i] for i in l]
    )  # decoder: take a list of integers, output a string

In [4]:

@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B, T, C = x.shape
        wei = self.nnet(x)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))  # (B, T, T)
        wei = F.softmax(wei, dim=-1)  # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x)  # (B,T,hs)
        out = wei @ v  # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out


class Head(nn.Module):
    """one head of self-attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B, T, C = x.shape
        k = self.key(x)  # (B,T,hs)
        q = self.query(x)  # (B,T,hs)
        # compute attention scores ("affinities")
        wei = (
            q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5
        )  # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))  # (B, T, T)
        wei = F.softmax(wei, dim=-1)  # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x)  # (B,T,hs)
        out = wei @ v  # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out


class MultiHeadAttention(nn.Module):
    """multiple heads of self-attention in parallel"""

    def __init__(self, num_heads, head_size, mlp_attention=False):
        super().__init__()

        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


class FeedFoward(nn.Module):
    """a simple linear layer followed by a non-linearity"""

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    """Transformer block: communication followed by computation"""

    def __init__(self, n_embd, n_head, mlp_attention=False):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size, mlp_attention=mlp_attention)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


class GPTLanguageModel(nn.Module):
    def __init__(self, mlp_attention=False):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(
            *[
                Block(n_embd, n_head=n_head, mlp_attention=mlp_attention)
                for _ in range(n_layer)
            ]
        )
        self.ln_f = nn.LayerNorm(n_embd)  # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx)  # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))  # (T,C)
        x = tok_emb + pos_emb  # (B,T,C)
        x = self.blocks(x)  # (B,T,C)
        x = self.ln_f(x)  # (B,T,C)
        logits = self.lm_head(x)  # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx

In [5]:
def next_word(word):
  context = torch.tensor(encode(word), dtype=torch.long, device=device).view(1,-1)
  print(decode(model.generate(context, max_new_tokens=10)[0].tolist()))

def write_poem(max_new_tokens):
  context = torch.zeros((1, 1), dtype=torch.long, device=device)
  print(decode(model.generate(context, max_new_tokens=max_new_tokens)[0].tolist()))


# Experiment

In [11]:
%%capture
#@title Select Poet
poet_name = 'khayyam' # @param ["ferdousi", "saadi", "hafez", "moulavi", "khayyam"]
path = f"{poet_name}.pth"
# Define the file path from which you want to read the characters
file_path = f"{poet_name}_dictionary.pkl"
# Open the file in binary read mode
with open(file_path, 'rb') as file:
    # Use pickle.load() to read the list of characters from the file
    chars = pickle.load(file)

vocab_size = len(chars)
# create a mapping from characters to integers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
model = GPTLanguageModel()
model.load_state_dict(torch.load(path))  # Load the saved state dictionary
model = model.to(device)
model.eval()





In [12]:
word = 'صبح' # @param {type:"string"}
next_word("صبح")

صبح صبوح
صصرا


In [13]:
write_poem(1000)

	است
گردش خویش اگر و در خاک شده
ابر آمد و زار بر سر سبزه گریست
بی باده گلرنگ نمی شاید زیست
و آن سبزه که امروز تماشاگه توست
فردا همه از خاک خود تو بود شکست
 لفیقل پیشه دل و جانشیده است پدید آمده است
از آمدن نبود گردو را سود
وز رفتن من جلهان و جدام و جهان عمر ما چیست
عاشق از خرابید گدان و نام و کف دست
این دسته که بر گردن او می بینی
دستی ست که برگردن یاری بوده است
این کوزه گرد که آبین عذریم
بر چو آب چرخ فلک به حور سوییز و من سودا را
می نوش به امروز خوش است پیمانند
امعلوم نیک و بد ز بند کف دست
در و خبرا با ناله کوزه هر گوشم و نه آبنشنود
جامی که جهان کودکی آمده بار
تا چند حجصل عمر قضا نیکوتر فروباشد
وز خوردن و چو بیش اندر باز بیشت گفت
بر من زلف نشان نبودن غدار ببطی غلط گرف جویی
 آن که به لگد می لعل و زلفین ز میفان دست
از مهر که پیوست و بل کلبم و کاست
عیشید آن در پرده است و در خوش است
کف صنیم و به کله کله کام و یک نفس طوس بهشت
آن قصر که با عدل کجام هیچ معلوم نشد
هم دانها باز کند و خبراب
آیا که نوبت جوانی غم فتاش و خوش نهاین بیهود
گر عارقبزه که کوزه گویا و میاید شکن
احصل عمر دل دان ای ساقی
با